In [68]:
import csv
import xml.etree.ElementTree as ET
import time
import io
from tqdm import tqdm
import pandas as pd


In [79]:
DRUGBANK_DATABASES:list() = [
    (0,"DB 3.0","database/3.0/drugbank.xml",2013),
    (1,"DB 4.1","database/4.1/drugbank.xml",2014), #not sure about year
    (2,"DB 4.2","database/4.2/drugbank.xml",2015), #not sure about year
    (3,"DB 4.3","database/4.3/drugbank.xml",2015),
    (4,"DB 4.5","database/4.5.0/drugbank.xml",2016),
    (5," DB 5.0","database/5.0.0/drugbank.xml",2016),
    (6,"DB 5.0.1","database/5.0.1/drugbank.xml",2016),
    (7,"DB 5.0.2","database/5.0.2/drugbank.xml",2016),
    (8,"DB 5.0.3","database/5.0.3/drugbank.xml",2016),
    (9,"DB 5.0.4","database/5.0.4/full_database.xml",2017),
    (10,"DB 5.0.5","database/5.0.5/full_database.xml",2017),
    (11,"DB 5.0.6","database/5.0.6/full_database.xml",2017),
    (12,"DB 5.0.7","database/5.0.7/full_database.xml",2017),
    (13,"DB 5.0.8","database/5.0.8/full_database.xml",2017),
    (14,"DB 5.0.9","database/5.0.9/full_database.xml",2017), 
    (15,"DB 5.0.10","database/5.0.10/full_database.xml",2017),
    (16,"DB 5.0.11","database/5.0.11/full_database.xml",2017),
    (17,"DB 5.1.0","database/5.1.0/full_database.xml",2018),
    (18,"DB 5.1.1","database/5.1.1/full_database.xml",2018),
    (19,"DB 5.1.2","database/5.1.2/full_database.xml",2018),
    (20,"DB 5.1.3","database/5.1.3/full_database.xml",2019),
    (21,"DB 5.1.4","database/5.1.4/full_database.xml",2019),
    (22,"DB 5.1.5","database/5.1.5/full_database.xml",2020),
    (23,"DB 5.1.6","database/5.1.6/full_database.xml",2020),
    (24,"DB 5.1.7","database/5.1.7/full_database.xml",2020),
    (25,"DB 5.1.8","database/5.1.8/full_database.xml",2021)
]

In [80]:
def search_drug_in_list(drug_id, drug_list):
    for item in drug_list:
        if item[0]==drug_id:
            return True
    
    return False

In [83]:
def get_approved_drugs_interactions(db_entry):
    try:
        tree = ET.parse(db_entry[2])
        root = tree.getroot()
        group_focus = "approved"
        drugs = list(root)
        database_name = db_entry[1].replace(".","_")
        database_name_trim = database_name.replace(" ","")
        output_file = "database/approved_only/"+database_name_trim+"_approved_drugs.csv"
        with io.open(output_file, "w", encoding="utf-8", newline='') as csv_file3:
            results_writer_cd = csv.writer(csv_file3, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            drug_list = []
            print("Parsing {0}".format(db_entry[2]))
            for i in tqdm(range(len(drugs))):
            #for i in range(0, 10):
                drug = drugs[i]
                idDB = drug[0].text
                drug_interaction = []
                for idx,feature in enumerate(drug):
                    if 'name' in str(feature): # drug name
                        drug_name = drug[idx].text
                        #print(drug_name)
                    if 'groups' in str(feature): #type of drug
                        group_name = list(drug[idx])[0].text
                        #print(group_name)
                    if 'drug-interactions' in str(feature): #interaction other drugs
                        #drug_interaction_ids = ';'.join([di[0].text for di in list(drug[idx])])
                       # drug_interaction = ';'.join([di[0].text for di in list(drug[idx])])
                        drug_interaction=[di[0].text for di in list(drug[idx])]
                        #print(drug_interaction)
                    if 'atc-codes' in str(feature): #atc-codes
                        atc_names = list(drug[idx])
                        #print(group_name)
                if (group_focus in group_name) and (len(atc_names)>0):
                    drug_list.append((idDB, drug_name,group_name,drug_interaction))
            for drug in drug_list:
                interactions_list = drug[3]
                new_interaction_list = []
                for interaction in interactions_list:
                    if search_drug_in_list(interaction,drug_list)==True:
                        new_interaction_list.append(interaction)
                drug_interaction_text =';'.join(new_interaction_list)
                results_writer_cd.writerow([drug[0], drug[1], drug[2], drug_interaction_text])
    except:
        print("Failed to parse {0}".format(db_entry[2]))

In [84]:
for item in DRUGBANK_DATABASES:
    get_approved_drugs_interactions(item)

 11%|████████▎                                                                    | 832/7682 [00:00<00:00, 8257.11it/s]

Parsing database/3.0/drugbank.xml


 11%|████████▏                                                                    | 828/7759 [00:00<00:00, 8227.75it/s]

Parsing database/4.1/drugbank.xml


 16%|████████████▍                                                               | 1268/7759 [00:00<00:01, 6309.44it/s]

...

  2%|█▎                                                                          | 238/13580 [00:00<00:05, 2365.61it/s]

Parsing database/5.1.7/full_database.xml


  0%|▎                                                                             | 63/14315 [00:00<00:22, 625.60it/s]

Parsing database/5.1.8/full_database.xml


100%|██████████████████████████████████████████████████████████████████████████| 14315/14315 [00:06<00:00, 2120.55it/s]
